# Music Analysis

In [1]:
import re
import sys
import time
import math
import random
import spotipy
import requests
import itertools
import collections
import pandas as pd
from tqdm import tqdm
import spotipy.util as util
from bs4 import BeautifulSoup
from collections import defaultdict
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
def get_track_features(client_id = None, client_secret = None):
    if not client_id or not client_secret:
        print("Please add your client_id and client_secret")
        print("For more information, please visit ")
        print("https://developer.spotify.com/my-applications/#!/applications")
        return 
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, \
                                   client_secret = client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    # Get track info
    artist_name = []
    track_name = []
    track_id = []
    popularity = []
    years = []

    for year in tqdm(range(1960, 2020)):
        try:
            track_results = sp.search(q='year:{0}'.format(year), type='track', limit=50)
        except:
            break
        while track_results:
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                popularity.append(t['popularity'])
                years.append(str(year))
            if track_results['tracks']['next']:
                try:
                    track_results = sp.next(track_results['tracks'])
                except:
                    break
            else:
                track_results = None
    print("Get {0} songs from spotify".format(len(track_id)))
    # Get featrues
    feature_keys = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', \
                      'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature']

    audio_features = {key:[] for key in feature_keys}
    audio_features['artist_name'] = artist_name
    audio_features['track_name'] = track_name
    audio_features['track_id'] = track_id
    audio_features['popularity'] = popularity
    audio_features['year'] = years # list(itertools.chain(*[[i] * 10000 for i in range(1960, 2020)])) # years

    for i in tqdm(range(0, len(track_id), 50)):
        features = sp.audio_features(track_id[i: i + 50])  
        for feature in features:
            for key in feature_keys:
                if feature and key in feature:
                    audio_features[key].append(feature[key])
                else:
                    audio_features[key].append(None)
    
    # Save to csv file
    df_tracks = pd.DataFrame(audio_features)
    df_tracks.head()
    df_tracks.to_csv(r'audio_features.csv')

In [3]:
# We've already fetch all the data and corresponding features,
# and saved it in "./audio_features.csv"
# no need to run this function again
client_id = None
client_secret = None
get_track_features(client_id, client_secret)

Please add your client_id and client_secret
For more information, please visit 
https://developer.spotify.com/my-applications/#!/applications


In [4]:
df_tracks = pd.read_csv('audio_features.csv', index_col=0)
df_tracks.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_name,track_name,track_id,popularity,year
0,0.171,0.330,5.0,-9.699,1.0,0.0329,0.707,0.00381,0.302,0.315,174.431,182400.0,3.0,Etta James,At Last,4Hhv2vrOTy89HFRcjU3QOx,75,1960
1,0.508,0.287,1.0,-12.472,1.0,0.0523,0.764,0.00000,0.153,0.644,154.759,175987.0,4.0,Ella Fitzgerald,Sleigh Ride,4ukUoXLuFzMixyZyabSGc4,70,1960
2,0.579,0.502,8.0,-7.570,1.0,0.0513,0.733,0.00000,0.281,0.836,76.816,131733.0,4.0,Ella Fitzgerald,Frosty The Snowman,65irrLqfCMRiO3p87P4C0D,69,1960
3,0.553,0.291,4.0,-10.426,0.0,0.0301,0.878,0.00000,0.129,0.407,96.217,165560.0,4.0,Sam Cooke,You Send Me,0BFEyqJ9DJXS7gKg0Kj46R,66,1960
4,0.482,0.569,0.0,-5.897,1.0,0.0328,0.617,0.00000,0.155,0.609,70.843,165280.0,4.0,Sam Cooke,Bring It On Home To Me,4yjz1aazw6R8ZURpGbCAkp,64,1960


In [5]:
df_by_year = df_tracks.dropna().groupby('year').mean().reset_index()
df_by_year.head()

,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,popularity
0,1960,0.464192,0.313453,5.005300,-14.693014,0.715200,0.100909,0.809230,0.253353,0.222327,0.494803,110.969244,215520.814900,3.750900,5.448100
1,1961,0.474276,0.320147,4.948585,-14.829432,0.720816,0.107911,0.799795,0.258537,0.218472,0.493421,110.856446,222618.542363,3.752826,5.115635
2,1962,0.463054,0.330539,5.049005,-14.832738,0.719772,0.100380,0.782238,0.254475,0.225328,0.484301,111.210020,214440.192719,3.749175,6.244124
3,1963,0.478339,0.357190,5.090700,-13.994034,0.742600,0.078978,0.758253,0.238693,0.220458,0.526543,112.315946,202682.442800,3.769200,7.717000
4,1964,0.478219,0.371290,5.084208,-13.754991,0.747575,0.080756,0.746077,0.217783,0.228598,0.528866,113.116872,198727.756076,3.767377,8.243324


In [6]:
import plotly.graph_objects as go
from ipywidgets import widgets
from ipywidgets import interactive
import plotly.express as px

In [7]:
virtualize_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', \
                      'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature', 'popularity']

feature_box = widgets.Dropdown(options=virtualize_features,
                          value=virtualize_features[0],
                          description='Feature: ')

trace1 = go.Scatter(x=df_by_year['year'], y=df_by_year['danceability'])

g = go.FigureWidget(data=[trace1],
                   layout=go.Layout(
                        title=dict(
                            text='Music Features Over Years'
                        ),
                        xaxis=dict(
                            title="Year"
                        ),
                        yaxis=dict(
                            title="danceability"
                        ),
                    ))

def response(change):
    new_feature = change['new']
    g.layout.yaxis.title = new_feature
    g.data[0].y = df_by_year[new_feature]

feature_box.observe(response, names="value")

widgets.VBox([feature_box, g])

In [8]:
all_musicians = list(df_tracks['artist_name'].unique())
print(len(all_musicians))

53981


In [9]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/
# https://stackoverflow.com/questions/45171646/how-to-get-birth-location-using-wikipedia-api
def search_birthlocation(musician):
    from SPARQLWrapper import SPARQLWrapper, JSON
    endpoint_url = "https://query.wikidata.org/sparql"
    query = """SELECT DISTINCT ?item ?itemLabel ?birthLocation ?birthLocationLabel WHERE {{
              ?item (wdt:P31|wdt:P101|wdt:P106)/wdt:P279* wd:Q639669 ;
            rdfs:label "{0}"@en ;wdt:P19 ?birthLocation
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            }}""".format(musician)
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    try:
        query_result = sparql.query().convert()
    except:
        musician_birth['birthplace'].append("NaN")
        return
    if not query_result["results"]["bindings"]:
        musician_birth['birthplace'].append("NaN")
    else:
        result = query_result["results"]["bindings"][0]
        if 'birthLocationLabel' in result:
            musician_birth['birthplace'].append(result['birthLocationLabel']['value'])
        else:
            musician_birth['birthplace'].append("NaN")

def get_musician_birth_by_wikisql(run = False):
    if not run:
        return 
    musician_birth = {key:[] for key in ['name', 'birthplace']}

    for i in tqdm(all_musicians):
        time.sleep(0.1)
        musician_birth['name'].append(i)
        search_birthlocation(i)

    print(musician_birth)
    df_musician = pd.DataFrame(musician_birth)
    df_musician.head()
    df_musician.to_csv(r'musician_birth.csv', index=[0])
    
get_musician_birth_by_wikisql()

In [10]:
df_musician = pd.read_csv('musician_birth.csv', index_col=0)
df_musician.head()

,name,birthplace
0,Etta James,Los Angeles
1,Ella Fitzgerald,Newport News
2,Sam Cooke,NaN
3,Neil Sedaka,Brooklyn
4,Dean Martin,Steubenville


In [11]:
def get_musician_birth_by_wiki(run = False):
    if not run:
        return 
    null = 0
    find = 0
    for i, name in enumerate(df_musician.name):
        if i != 0 and i % 1000 == 0:
            print(i)
            print(find)
            print(null)
        if i < 34369:
            continue
        if type(df_musician.loc[i, "birthplace"]) != float or not math.isnan(df_musician.loc[i, "birthplace"]):
            continue
        null += 1
        url = "https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=xmlfm&titles={0}&rvsection=0".format(name)
        r = requests.get(url)
        soup = BeautifulSoup(r._content, 'html.parser')
        all_matches = re.findall(r"birth_place [^\n]+\n\|", soup.prettify())
        if len(all_matches) == 0:
            all_matches = re.findall(r"origin [^\n]+\n\|", soup.prettify())
        if len(all_matches):
            df_musician.loc[i, "birthplace"] = all_matches[0].strip('\n|').replace('[', '').replace(']', '').split('=')[1].strip()
            find += 1
        time.sleep(random.uniform(0.1, 0.4))
        df_musician.to_csv(r'musician_birth1.csv', index=[0])

get_musician_birth_by_wiki()

In [52]:
df_musician = pd.read_csv('musician_birth1.csv', index_col=0)
df_musician.head()

,name,birthplace
0,Etta James,Los Angeles
1,Ella Fitzgerald,Newport News
2,Sam Cooke,"Clarksdale, Mississippi, U.S."
3,Neil Sedaka,Brooklyn
4,Dean Martin,Steubenville


In [13]:
print(len(df_musician[df_musician['birthplace'].isnull()]))

33403


In [14]:
df_musician.head()

,name,birthplace
0,Etta James,Los Angeles
1,Ella Fitzgerald,Newport News
2,Sam Cooke,"Clarksdale, Mississippi, U.S."
3,Neil Sedaka,Brooklyn
4,Dean Martin,Steubenville


In [20]:
new_df = df_musician[df_musician['birthplace'].isnull()]
l = set(list(new_df.name))
print(len(df_tracks[df_tracks['artist_name'].apply(lambda x : x not in l)]))
df_tracks[df_tracks['artist_name'].apply(lambda x : x not in l)].dropna().groupby('year').count()

400205


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_name,track_name,track_id,popularity
year,,,,,,,,,,,,,,,,,
1960,6652,6652,6652,6652,6652,6652,6652,6652,6652,6652,6652,6652,6652,6652,6652,6652,6652
1961,6533,6533,6533,6533,6533,6533,6533,6533,6533,6533,6533,6533,6533,6533,6533,6533,6533
1962,6971,6971,6971,6971,6971,6971,6971,6971,6971,6971,6971,6971,6971,6971,6971,6971,6971
1963,6991,6991,6991,6991,6991,6991,6991,6991,6991,6991,6991,6991,6991,6991,6991,6991,6991
1964,6850,6850,6850,6850,6850,6850,6850,6850,6850,6850,6850,6850,6850,6850,6850,6850,6850
1965,7083,7083,7083,7083,7083,7083,7083,7083,7083,7083,7083,7083,7083,7083,7083,7083,7083
1966,7095,7095,7095,7095,7095,7095,7095,7095,7095,7095,7095,7095,7095,7095,7095,7095,7095
1967,7464,7464,7464,7464,7464,7464,7464,7464,7464,7464,7464,7464,7464,7464,7464,7464,7464
1968,7198,7198,7198,7198,7198,7198,7198,7198,7198,7198,7198,7198,7198,7198,7198,7198,7198


In [68]:
df = df_tracks[df_tracks['artist_name'].apply(lambda x : x in l)]
df = df.groupby('artist_name').size().sort_values()

In [69]:
df = df > 10

df = df[df]

In [79]:
print(len(df.index))

names = ([name for name in df.index if name not in res])
print(len(names))

3633
2962


In [83]:
singer_res = {}
def get_musician_birth_by_wiki(run = False):
    if not run:
        return 
    null = 0
    find = 0
    for name in tqdm(names):
        null += 1
        url = "https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=xmlfm&titles={0}&rvsection=0".format(name + " (singer)")
        r = requests.get(url)
        soup = BeautifulSoup(r._content, 'html.parser')
        all_matches = re.findall(r"birth_place [^\n]+\n\|", soup.prettify())
        if len(all_matches) == 0:
            all_matches = re.findall(r"origin [^\n]+\n\|", soup.prettify())
        if len(all_matches):
            singer_res[name] = all_matches[0].strip('\n|').replace('[', '').replace(']', '').split('=')[1].strip()
            find += 1
            if find % 10 == 0:
                print(find)
                print(null)
        time.sleep(random.uniform(0.1, 0.4))
                                                                                                                                        
get_musician_birth_by_wiki(run = True)                                                                                                                         



  0%|                                                                                         | 0/2962 [00:00<?, ?it/s]

  0%|                                                                                 | 1/2962 [00:00<31:43,  1.56it/s]

  0%|                                                                                 | 2/2962 [00:01<32:36,  1.51it/s]

  0%|                                                                                 | 3/2962 [00:02<32:30,  1.52it/s]

  0%|                                                                                 | 4/2962 [00:02<30:34,  1.61it/s]

  0%|▏                                                                                | 5/2962 [00:03<30:01,  1.64it/s]

  0%|▏                                                                                | 6/2962 [00:03<29:10,  1.69it/s]

  0%|▏                                                                                | 7/2962 [00:04<33:40,  1.46it/s]

  0%|▏                        

  2%|█▊                                                                              | 67/2962 [00:57<41:30,  1.16it/s]

  2%|█▊                                                                              | 68/2962 [00:58<38:41,  1.25it/s]

  2%|█▊                                                                              | 69/2962 [00:59<42:21,  1.14it/s]

  2%|█▉                                                                              | 70/2962 [01:00<43:45,  1.10it/s]

  2%|█▉                                                                              | 71/2962 [01:01<45:28,  1.06it/s]

  2%|█▉                                                                              | 72/2962 [01:02<42:27,  1.13it/s]

  2%|█▉                                                                              | 73/2962 [01:02<40:04,  1.20it/s]

  2%|█▉                                                                              | 74/2962 [01:03<40:30,  1.19it/s]

  3%|██                         

  5%|███▌                                                                           | 134/2962 [01:56<39:40,  1.19it/s]

  5%|███▌                                                                           | 135/2962 [01:57<38:32,  1.22it/s]

  5%|███▋                                                                           | 136/2962 [01:58<38:55,  1.21it/s]

  5%|███▋                                                                           | 137/2962 [01:58<41:15,  1.14it/s]

  5%|███▋                                                                           | 138/2962 [01:59<40:31,  1.16it/s]

  5%|███▋                                                                           | 139/2962 [02:00<40:09,  1.17it/s]

  5%|███▋                                                                           | 140/2962 [02:01<37:44,  1.25it/s]

  5%|███▊                                                                           | 141/2962 [02:02<42:30,  1.11it/s]

  5%|███▊                       

  7%|█████▎                                                                         | 201/2962 [02:58<41:52,  1.10it/s]

  7%|█████▍                                                                         | 202/2962 [02:59<41:58,  1.10it/s]

  7%|█████▍                                                                         | 203/2962 [03:00<42:50,  1.07it/s]

  7%|█████▍                                                                         | 204/2962 [03:01<40:56,  1.12it/s]

  7%|█████▍                                                                         | 205/2962 [03:02<45:04,  1.02it/s]

  7%|█████▍                                                                         | 206/2962 [03:03<43:11,  1.06it/s]

  7%|█████▌                                                                         | 207/2962 [03:04<43:14,  1.06it/s]

  7%|█████▌                                                                         | 208/2962 [03:05<43:47,  1.05it/s]

  7%|█████▌                     

  9%|███████▏                                                                       | 268/2962 [03:56<37:07,  1.21it/s]

  9%|███████▏                                                                       | 269/2962 [03:57<36:44,  1.22it/s]

  9%|███████▏                                                                       | 270/2962 [03:58<34:54,  1.29it/s]

  9%|███████▏                                                                       | 271/2962 [03:59<34:55,  1.28it/s]

  9%|███████▎                                                                       | 272/2962 [04:00<36:59,  1.21it/s]

  9%|███████▎                                                                       | 273/2962 [04:00<37:23,  1.20it/s]

  9%|███████▎                                                                       | 274/2962 [04:01<37:49,  1.18it/s]

  9%|███████▎                                                                       | 275/2962 [04:02<38:35,  1.16it/s]

  9%|███████▎                   

...

 91%|██████████████████████████████████████████████████████████████████████▋       | 2683/2962 [39:33<04:07,  1.13it/s]

 91%|██████████████████████████████████████████████████████████████████████▋       | 2684/2962 [39:34<03:56,  1.18it/s]

 91%|██████████████████████████████████████████████████████████████████████▋       | 2685/2962 [39:35<03:38,  1.27it/s]

 91%|██████████████████████████████████████████████████████████████████████▋       | 2686/2962 [39:36<03:34,  1.29it/s]

 91%|██████████████████████████████████████████████████████████████████████▊       | 2687/2962 [39:37<03:46,  1.21it/s]

 91%|██████████████████████████████████████████████████████████████████████▊       | 2688/2962 [39:37<03:57,  1.15it/s]

 91%|██████████████████████████████████████████████████████████████████████▊       | 2689/2962 [39:38<03:51,  1.18it/s]

 91%|██████████████████████████████████████████████████████████████████████▊       | 2690/2962 [39:39<03:49,  1.18it/s]

 91%|███████████████████████████

 93%|████████████████████████████████████████████████████████████████████████▍     | 2750/2962 [40:38<04:26,  1.26s/it]

 93%|████████████████████████████████████████████████████████████████████████▍     | 2751/2962 [40:40<05:19,  1.52s/it]

 93%|████████████████████████████████████████████████████████████████████████▍     | 2752/2962 [40:42<04:53,  1.40s/it]

 93%|████████████████████████████████████████████████████████████████████████▍     | 2753/2962 [40:42<04:11,  1.21s/it]

 93%|████████████████████████████████████████████████████████████████████████▌     | 2754/2962 [40:43<03:46,  1.09s/it]

 93%|████████████████████████████████████████████████████████████████████████▌     | 2755/2962 [40:44<03:42,  1.08s/it]

 93%|████████████████████████████████████████████████████████████████████████▌     | 2756/2962 [40:45<03:20,  1.03it/s]

 93%|████████████████████████████████████████████████████████████████████████▌     | 2757/2962 [40:46<03:49,  1.12s/it]

 93%|███████████████████████████

 95%|██████████████████████████████████████████████████████████████████████████▏   | 2817/2962 [41:47<02:08,  1.13it/s]

 95%|██████████████████████████████████████████████████████████████████████████▏   | 2818/2962 [41:48<02:05,  1.14it/s]

 95%|██████████████████████████████████████████████████████████████████████████▏   | 2819/2962 [41:49<02:09,  1.10it/s]

 95%|██████████████████████████████████████████████████████████████████████████▎   | 2820/2962 [41:49<01:59,  1.19it/s]

 95%|██████████████████████████████████████████████████████████████████████████▎   | 2821/2962 [41:50<01:55,  1.22it/s]

 95%|██████████████████████████████████████████████████████████████████████████▎   | 2822/2962 [41:51<01:53,  1.23it/s]

 95%|██████████████████████████████████████████████████████████████████████████▎   | 2823/2962 [41:52<01:49,  1.28it/s]

 95%|██████████████████████████████████████████████████████████████████████████▎   | 2824/2962 [41:52<01:48,  1.28it/s]

 95%|███████████████████████████

 97%|███████████████████████████████████████████████████████████████████████████▉  | 2884/2962 [42:42<01:07,  1.15it/s]

 97%|███████████████████████████████████████████████████████████████████████████▉  | 2885/2962 [42:43<01:01,  1.25it/s]

 97%|███████████████████████████████████████████████████████████████████████████▉  | 2886/2962 [42:43<00:57,  1.31it/s]

 97%|████████████████████████████████████████████████████████████████████████████  | 2887/2962 [42:44<00:57,  1.31it/s]

 98%|████████████████████████████████████████████████████████████████████████████  | 2888/2962 [42:45<00:57,  1.29it/s]

 98%|████████████████████████████████████████████████████████████████████████████  | 2889/2962 [42:46<00:55,  1.31it/s]

 98%|████████████████████████████████████████████████████████████████████████████  | 2890/2962 [42:47<00:59,  1.21it/s]

 98%|████████████████████████████████████████████████████████████████████████████▏ | 2891/2962 [42:47<00:55,  1.28it/s]

 98%|███████████████████████████

100%|█████████████████████████████████████████████████████████████████████████████▋| 2951/2962 [43:39<00:09,  1.18it/s]

100%|█████████████████████████████████████████████████████████████████████████████▋| 2952/2962 [43:40<00:08,  1.11it/s]

100%|█████████████████████████████████████████████████████████████████████████████▊| 2953/2962 [43:41<00:07,  1.15it/s]

100%|█████████████████████████████████████████████████████████████████████████████▊| 2954/2962 [43:42<00:07,  1.12it/s]

100%|█████████████████████████████████████████████████████████████████████████████▊| 2955/2962 [43:42<00:05,  1.20it/s]

100%|█████████████████████████████████████████████████████████████████████████████▊| 2956/2962 [43:43<00:05,  1.18it/s]

100%|█████████████████████████████████████████████████████████████████████████████▊| 2957/2962 [43:44<00:04,  1.17it/s]

100%|█████████████████████████████████████████████████████████████████████████████▉| 2958/2962 [43:45<00:03,  1.20it/s]

100%|███████████████████████████

In [84]:
print(len(singer_res))
import json
with open('singer_res.json', 'w') as outfile:
    json.dump(singer_res, outfile)

38


In [88]:
with open('band_res.json', 'r') as outfile:
    res = json.load(outfile)
print(len(res))

671


In [89]:
res.update(singer_res)

In [90]:
print(len(res))

709


In [92]:
df_musician = pd.read_csv('musician_birth1.csv', index_col=0)
df_musician.head()

,name,birthplace
0,Etta James,Los Angeles
1,Ella Fitzgerald,Newport News
2,Sam Cooke,"Clarksdale, Mississippi, U.S."
3,Neil Sedaka,Brooklyn
4,Dean Martin,Steubenville


In [96]:
for i, name in enumerate(df_musician.name):
    if name in res:
        df_musician.loc[i, 'birthplace'] = res[name]

In [101]:
print(len(df_musician[df_musician['birthplace'].isnull()]))

32694


In [98]:
new_df = df_musician[df_musician['birthplace'].isnull()]
l = set(list(new_df.name))
print(len(df_tracks[df_tracks['artist_name'].apply(lambda x : x not in l)]))
df_tracks[df_tracks['artist_name'].apply(lambda x : x not in l)].dropna().groupby('year').count()

432396


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_name,track_name,track_id,popularity
year,,,,,,,,,,,,,,,,,
1960,6758,6758,6758,6758,6758,6758,6758,6758,6758,6758,6758,6758,6758,6758,6758,6758,6758
1961,6678,6678,6678,6678,6678,6678,6678,6678,6678,6678,6678,6678,6678,6678,6678,6678,6678
1962,7011,7011,7011,7011,7011,7011,7011,7011,7011,7011,7011,7011,7011,7011,7011,7011,7011
1963,7143,7143,7143,7143,7143,7143,7143,7143,7143,7143,7143,7143,7143,7143,7143,7143,7143
1964,6972,6972,6972,6972,6972,6972,6972,6972,6972,6972,6972,6972,6972,6972,6972,6972,6972
1965,7202,7202,7202,7202,7202,7202,7202,7202,7202,7202,7202,7202,7202,7202,7202,7202,7202
1966,7165,7165,7165,7165,7165,7165,7165,7165,7165,7165,7165,7165,7165,7165,7165,7165,7165
1967,7598,7598,7598,7598,7598,7598,7598,7598,7598,7598,7598,7598,7598,7598,7598,7598,7598
1968,7525,7525,7525,7525,7525,7525,7525,7525,7525,7525,7525,7525,7525,7525,7525,7525,7525


In [102]:
df_musician.to_csv(r'musician_birth_final.csv', index=[0])